In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus
import requests

import urllib.request as ul
import xmltodict
import json
import sys
import io

In [ ]:
def make_locdate(x):
    Ymd, HMS = x.split(' ')
    Y, M, D = Ymd.split('-')
    return Y+M+D

# 풍력: 광주
전남 화순 동면 일출일몰 시각이 없어서 광주 데이터 사용

In [ ]:
wind = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/wind_forecast_f.csv')
wind

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-29 00:00:00,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0
1,2020-08-29 01:00:00,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0
2,2020-08-29 02:00:00,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0
3,2020-08-29 03:00:00,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0
4,2020-08-29 04:00:00,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0
16124,2022-07-01 20:00:00,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0
16125,2022-07-01 21:00:00,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0
16126,2022-07-01 22:00:00,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0


In [ ]:
wind['datetime'] = wind['datetime'].astype(str) 
wind['locdate'] = wind['datetime'].apply(lambda x:make_locdate(x))
wind

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate
0,2020-08-29 00:00:00,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0,20200829
1,2020-08-29 01:00:00,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0,20200829
2,2020-08-29 02:00:00,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0,20200829
3,2020-08-29 03:00:00,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0,20200829
4,2020-08-29 04:00:00,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0,20200829
...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0,20220701
16124,2022-07-01 20:00:00,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0,20220701
16125,2022-07-01 21:00:00,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0,20220701
16126,2022-07-01 22:00:00,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0,20220701


In [ ]:
url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'

for i in range(0, wind.shape[0], 24):
    locdate = wind.loc[i,'locdate']
    params ={'serviceKey' : '0GE5dffPQ2w31OZOACOdSgFRQzPihrAaslHTL1WR8UajH2KJivTGQfGkmw8pCFaWIr9sG2Vh4YEuaRk4rvCgSg==', 'locdate' : locdate, 'location' : '광주' }
    response = requests.get(url, params)
    resp = xmltodict.parse(response.text)
    print(resp)
    wind.loc[i, 'sunrise'] = resp['response']['body']['items']['item']['sunrise']
    wind.loc[i, 'sunset'] = resp['response']['body']['items']['item']['sunset']
    print(i)
print('end')

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'aste': '2032', 'astm': '0434', 'civile': '1930', 'civilm': '0537', 'latitude': '3509', 'latitudeNum': '35.1500000', 'location': '광주', 'locdate': '20200829', 'longitude': '12650', 'longitudeNum': '126.8333330', 'moonrise': '1646', 'moonset': '0145', 'moontransit': '2143', 'naute': '2000', 'nautm': '0506', 'sunrise': '0603', 'sunset': '1904', 'suntransit': '123332'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}}
0
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'aste': '2031', 'astm': '0435', 'civile': '1928', 'civilm': '0538', 'latitude': '3509', 'latitudeNum': '35.1500000', 'location': '광주', 'locdate': '20200830', 'longitude': '12650', 'longitudeNum': '126.8333330', 'moonrise': '1733', 'moonset': '0243', 'moontransit': '2237', 'naute': '1959', 'nautm': '0507', 'sunrise': '0604', 'sunset': '1902', 'suntransit': '12

In [ ]:
for i in range(0, wind.shape[0], 24):
    wind['sunrise'] = wind['sunrise'].fillna(method='ffill')
    wind['sunset'] = wind['sunset'].fillna(method='ffill')

In [ ]:
wind

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate,sunrise,sunset
0,2020-08-29 00:00:00,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0,20200829,0603,1904
1,2020-08-29 01:00:00,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0,20200829,0603,1904
2,2020-08-29 02:00:00,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0,20200829,0603,1904
3,2020-08-29 03:00:00,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0,20200829,0603,1904
4,2020-08-29 04:00:00,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0,20200829,0603,1904
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0,20220701,0522,1951
16124,2022-07-01 20:00:00,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0,20220701,0522,1951
16125,2022-07-01 21:00:00,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0,20220701,0522,1951
16126,2022-07-01 22:00:00,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0,20220701,0522,1951


In [ ]:
wind['sunrise'] = wind['sunrise'].astype(int)//100
wind['sunset'] = wind['sunset'].astype(int)//100

In [ ]:
wind['sunshine']= wind["sunset"] - wind["sunrise"]
wind

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0,20200829,6,19,13
1,2020-08-29 01:00:00,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0,20200829,6,19,13
2,2020-08-29 02:00:00,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0,20200829,6,19,13
3,2020-08-29 03:00:00,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0,20200829,6,19,13
4,2020-08-29 04:00:00,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0,20200829,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0,20220701,5,19,14
16124,2022-07-01 20:00:00,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0,20220701,5,19,14
16125,2022-07-01 21:00:00,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0,20220701,5,19,14
16126,2022-07-01 22:00:00,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0,20220701,5,19,14


In [ ]:
wind1 = wind.copy()
wind1 = wind1.drop(['locdate'], axis=1)

In [ ]:
wind1

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0,6,19,13
1,2020-08-29 01:00:00,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0,6,19,13
2,2020-08-29 02:00:00,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0,6,19,13
3,2020-08-29 03:00:00,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0,6,19,13
4,2020-08-29 04:00:00,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0,5,19,14
16124,2022-07-01 20:00:00,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0,5,19,14
16125,2022-07-01 21:00:00,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0,5,19,14
16126,2022-07-01 22:00:00,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0,5,19,14


In [ ]:
wind1.to_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/wind_forecast_f_sun.csv', index=False)

# 태양광: 인천(청라3동)

In [ ]:
#solar2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/에너지 인공지능 경진대회/forecast/new_data/closer/solar_forecast_total.csv')
#solar2

In [ ]:
solar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/solar_forecast_f.csv')
solar

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow
0,2020-08-29 00:00:00,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0
1,2020-08-29 01:00:00,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0
2,2020-08-29 02:00:00,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0
3,2020-08-29 03:00:00,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0
4,2020-08-29 04:00:00,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0
16124,2022-07-01 20:00:00,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0
16125,2022-07-01 21:00:00,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0
16126,2022-07-01 22:00:00,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0


In [ ]:
solar['datetime'] = solar['datetime'].astype(str) 
solar['locdate'] = solar['datetime'].apply(lambda x:make_locdate(x))
solar

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate
0,2020-08-29 00:00:00,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0,20200829
1,2020-08-29 01:00:00,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0,20200829
2,2020-08-29 02:00:00,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0,20200829
3,2020-08-29 03:00:00,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0,20200829
4,2020-08-29 04:00:00,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0,20200829
...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0,20220701
16124,2022-07-01 20:00:00,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0,20220701
16125,2022-07-01 21:00:00,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0,20220701
16126,2022-07-01 22:00:00,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0,20220701


In [ ]:
url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'

for i in range(0, solar.shape[0], 24):
    locdate = solar.loc[i,'locdate']
    params ={'serviceKey' : '0GE5dffPQ2w31OZOACOdSgFRQzPihrAaslHTL1WR8UajH2KJivTGQfGkmw8pCFaWIr9sG2Vh4YEuaRk4rvCgSg==', 'locdate' : locdate, 'location' : '인천' }
    response = requests.get(url, params)
    resp = xmltodict.parse(response.text)
    print(resp)
    solar.loc[i, 'sunrise'] = resp['response']['body']['items']['item']['sunrise']
    solar.loc[i, 'sunset'] = resp['response']['body']['items']['item']['sunset']
    print(i)
print('end')

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'aste': '2038', 'astm': '0429', 'civile': '1933', 'civilm': '0534', 'latitude': '3727', 'latitudeNum': '37.4500000', 'location': '인천', 'locdate': '20200829', 'longitude': '12642', 'longitudeNum': '126.7000000', 'moonrise': '1653', 'moonset': '0138', 'moontransit': '2144', 'naute': '2005', 'nautm': '0502', 'sunrise': '0601', 'sunset': '1907', 'suntransit': '123406'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}}
0
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'aste': '2037', 'astm': '0430', 'civile': '1932', 'civilm': '0535', 'latitude': '3727', 'latitudeNum': '37.4500000', 'location': '인천', 'locdate': '20200830', 'longitude': '12642', 'longitudeNum': '126.7000000', 'moonrise': '1740', 'moonset': '0237', 'moontransit': '2237', 'naute': '2004', 'nautm': '0503', 'sunrise': '0602', 'sunset': '1905', 'suntransit': '12

In [ ]:
for i in range(0, solar.shape[0], 24):
    solar['sunrise'] = solar['sunrise'].fillna(method='ffill')
    solar['sunset'] = solar['sunset'].fillna(method='ffill')

In [ ]:
solar

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate,sunrise,sunset
0,2020-08-29 00:00:00,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0,20200829,0601,1907
1,2020-08-29 01:00:00,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0,20200829,0601,1907
2,2020-08-29 02:00:00,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0,20200829,0601,1907
3,2020-08-29 03:00:00,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0,20200829,0601,1907
4,2020-08-29 04:00:00,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0,20200829,0601,1907
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0,20220701,0516,1958
16124,2022-07-01 20:00:00,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0,20220701,0516,1958
16125,2022-07-01 21:00:00,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0,20220701,0516,1958
16126,2022-07-01 22:00:00,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0,20220701,0516,1958


In [ ]:
solar['sunrise'] = solar['sunrise'].astype(int)//100
solar['sunset'] = solar['sunset'].astype(int)//100

In [ ]:
solar['sunshine']= solar["sunset"] - solar["sunrise"]
solar

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,locdate,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0,20200829,6,19,13
1,2020-08-29 01:00:00,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0,20200829,6,19,13
2,2020-08-29 02:00:00,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0,20200829,6,19,13
3,2020-08-29 03:00:00,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0,20200829,6,19,13
4,2020-08-29 04:00:00,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0,20200829,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0,20220701,5,19,14
16124,2022-07-01 20:00:00,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0,20220701,5,19,14
16125,2022-07-01 21:00:00,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0,20220701,5,19,14
16126,2022-07-01 22:00:00,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0,20220701,5,19,14


In [ ]:
solar1 = solar.copy()
solar1 = solar1.drop(['locdate'], axis=1)

In [ ]:
solar1

,datetime,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0,6,19,13
1,2020-08-29 01:00:00,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0,6,19,13
2,2020-08-29 02:00:00,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0,6,19,13
3,2020-08-29 03:00:00,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0,6,19,13
4,2020-08-29 04:00:00,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0,5,19,14
16124,2022-07-01 20:00:00,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0,5,19,14
16125,2022-07-01 21:00:00,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0,5,19,14
16126,2022-07-01 22:00:00,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0,5,19,14


In [ ]:
solar1.to_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/solar_forecast_f_sun.csv', index=False)

In [ ]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           16128 non-null  object 
 1   temperature        16128 non-null  float64
 2   precipitationform  16128 non-null  float64
 3   precipitationprob  16128 non-null  float64
 4   humidity           16128 non-null  float64
 5   windspeed          16128 non-null  float64
 6   winddirection      16128 non-null  float64
 7   cloud              16128 non-null  float64
 8   precipitation      16128 non-null  float64
 9   snow               16128 non-null  float64
 10  locdate            16128 non-null  object 
 11  sunrise            16128 non-null  int64  
 12  sunset             16128 non-null  int64  
 13  sunshine           16128 non-null  int64  
dtypes: float64(9), int64(3), object(2)
memory usage: 1.7+ MB
